1) trendlib/config.py
Purpose: Centralize user-defined parameters (dates, frequency, toggles, file paths, etc.).

<details> <summary><strong>Click to show code</strong></summary>

In [ ]:
# trendlib/config.py

from datetime import datetime
from typing import Dict, Any

DEFAULT_CONFIG: Dict[str, Any] = {
    # Paths
    "data_file": "Data/YourData.csv",  # NOTE: Adjust if different
    "date_column": "Date",            # If your code uses "Date"
    "manager_column": "Ticker",       # NOTE: If you have "Ticker" or "Symbol"

    # Analysis window
    "analysis_start_date": "2000-01-01",
    "analysis_end_date":   "2020-12-31",

    # Frequency / Rolling windows (Phase 2 usage)
    "in_sample_length":  12,  # months? days? reference your code
    "out_sample_length": 6,

    # Volatility toggle
    "volatility_adjusted": True,

    # Demo / Test Mode
    "demo_mode": False,
    "demo_start_date": "2019-01-01",
    "demo_end_date":   "2019-06-30",
    "demo_num_managers": 5,

    # Export Paths
    "excel_path": "output/Trend_Analysis.xlsx",
    "csv_path":   "output/Trend_Analysis.csv",
    "json_path":  "output/Trend_Analysis.json"
}

def validate_config(cfg: Dict[str, Any]) -> None:
    """
    Light checks to ensure logic consistency.
    """
    try:
        # Check date ordering
        start = datetime.fromisoformat(cfg["analysis_start_date"])
        end   = datetime.fromisoformat(cfg["analysis_end_date"])
        if start >= end:
            raise ValueError("analysis_start_date must be before analysis_end_date.")

        if cfg["demo_mode"]:
            ds = datetime.fromisoformat(cfg["demo_start_date"])
            de = datetime.fromisoformat(cfg["demo_end_date"])
            if ds >= de:
                raise ValueError("demo_start_date must be before demo_end_date.")

    except KeyError as ke:
        raise ValueError(f"Missing config parameter: {ke}")


2) trendlib/data_io.py
Purpose: Load, subset, and clean the data. Integrates your logic from “Load Data” and “Prep Data” steps in the notebook.

<details> <summary><strong>Click to show code</strong></summary>

In [ ]:
# trendlib/data_io.py

import pandas as pd
import random
from typing import Dict
from trendlib.config import DEFAULT_CONFIG

def load_data(config: Dict = DEFAULT_CONFIG) -> pd.DataFrame:
    """
    Reads the main dataset from CSV, sets index to date, 
    filters to [analysis_start_date, analysis_end_date].
    If demo_mode=True, further subsets date & managers.
    """
    # 1. Read CSV
    df = pd.read_csv(
        config["data_file"],
        parse_dates=[config["date_column"]]
    )
    
    df.set_index(config["date_column"], inplace=True, drop=True)

    # 2. Limit by analysis start/end
    start_date = config["analysis_start_date"]
    end_date   = config["analysis_end_date"]
    df = df.loc[start_date:end_date]

    # 3. Basic cleaning from the original notebook logic
    # NOTE: Insert your original "dropna" or renaming code as needed.
    # For instance:
    # df.dropna(subset=["Close"], inplace=True)
    # or reference "Return" columns from your code.

    # 4. Demo/test mode subsetting
    if config.get("demo_mode", False):
        demo_start = config["demo_start_date"]
        demo_end   = config["demo_end_date"]
        df = df.loc[demo_start:demo_end]

        manager_col = config["manager_column"]
        all_managers = df[manager_col].unique().tolist()
        num_m = min(config["demo_num_managers"], len(all_managers))
        chosen = random.sample(all_managers, num_m)
        df = df[df[manager_col].isin(chosen)]

        print(f"[Demo Mode] Data from {demo_start} to {demo_end}, "
              f"{num_m} managers chosen out of {len(all_managers)} total.")

    return df


3) trendlib/metrics.py
Purpose: Move your performance calculations (e.g., volatility, returns, Sharpe, drawdown) here.

<details> <summary><strong>Click to show code</strong></summary>
python
Copy


In [ ]:
# trendlib/metrics.py

import pandas as pd
import numpy as np
from typing import Optional

def calculate_returns(df: pd.DataFrame, price_col: str = "Close") -> pd.Series:
    """
    Example placeholder: compute daily returns from a 'Close' column.
    Adjust to match your actual logic from the notebook.
    """
    return df[price_col].pct_change().fillna(0)


def annualized_volatility(returns: pd.Series, periods_per_year=252) -> float:
    """
    Standard annualized volatility = std(returns) * sqrt(periods_per_year).
    """
    if returns.empty:
        return np.nan
    return returns.std() * np.sqrt(periods_per_year)


def sharpe_ratio(returns: pd.Series, risk_free_annual=0.0, periods_per_year=252) -> float:
    if returns.empty:
        return np.nan
    # Convert annual risk-free to each period
    rf_period = risk_free_annual / periods_per_year
    excess = returns - rf_period
    vol = annualized_volatility(excess, periods_per_year)
    if vol == 0:
        return np.nan
    return (excess.mean() * periods_per_year) / vol


def max_drawdown(returns: pd.Series) -> float:
    """
    If you track cumulative returns, compute max drawdown.
    Insert your code from notebook.
    """
    if returns.empty:
        return np.nan
    cum = (1 + returns).cumprod()
    peak = cum.cummax()
    drawdown = (cum - peak) / peak
    return drawdown.min()


(Customize these to match your real variable names and logic from the notebook.)

4) trendlib/exports.py
Purpose: Provide consistent, try/except-wrapped export to Excel/CSV/JSON.

<details> <summary><strong>Click to show code</strong></summary>

In [ ]:
# trendlib/exports.py

import pandas as pd

def to_excel(df: pd.DataFrame, path: str) -> None:
    """
    Write the DataFrame to Excel with minimal error handling.
    """
    try:
        with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
            df.to_excel(writer, sheet_name='TrendAnalysis', index=False)
            # Optional formatting or freeze panes
            ws = writer.sheets['TrendAnalysis']
            ws.freeze_panes(1, 0)
    except PermissionError:
        print(f"[Export] Permission denied for '{path}'. Close file if open.")
    except Exception as e:
        print(f"[Export] Error writing Excel: {e}")


def to_csv(df: pd.DataFrame, path: str) -> None:
    try:
        df.to_csv(path, index=False)
    except Exception as e:
        print(f"[Export] CSV write error: {e}")


def to_json(df: pd.DataFrame, path: str) -> None:
    try:
        df.to_json(path, orient="records", indent=2)
    except Exception as e:
        print(f"[Export] JSON write error: {e}")


5) trendlib/selection.py (Optional)
Purpose: If you have a rank-ordered or manual selection approach, place that logic here for modularity.

<details> <summary><strong>Click to show code</strong></summary>

In [ ]:
# trendlib/selection.py

import pandas as pd

def rank_ordered_selection(df: pd.DataFrame, metric_col: str, top_n: int = 5) -> pd.DataFrame:
    """
    Sort by 'metric_col' descending, pick top N rows. 
    Example logic only; adapt to your actual code.
    """
    sorted_df = df.sort_values(metric_col, ascending=False)
    return sorted_df.head(top_n)


6) Your New, Simplified Notebook: Vol_Adj_Trend_Analysis1.2.TrEx.ipynb
Below is an example skeleton of how the notebook might look after the refactor. You’ll likely have more analysis steps, but this shows how to import from trendlib/ and reuse the code.

<details> <summary><strong>Click to show code</strong></summary>

In [ ]:
# Cell 1: Imports
import pandas as pd
import numpy as np

from trendlib.config import DEFAULT_CONFIG, validate_config
from trendlib.data_io import load_data
from trendlib.metrics import calculate_returns, annualized_volatility, sharpe_ratio, max_drawdown
from trendlib.exports import to_excel, to_csv, to_json
# from trendlib.selection import rank_ordered_selection  # if needed

# Cell 2: Setup / Config
config = DEFAULT_CONFIG.copy()
# Tweak any parameters you want:
config["data_file"] = "Data/Vol_Adj_Trend_Sample.csv"  # Example path
config["demo_mode"] = False
validate_config(config)

# Cell 3: Data Load
df = load_data(config)
print("Data shape:", df.shape)
print(df.head())

# Cell 4: Basic Single-Window Analysis
# Insert your original logic. For example, if you have in_sample vs out_sample splitting:
in_sample_df = df.loc["2008-01-01":"2012-12-31"]
out_sample_df = df.loc["2013-01-01":"2015-12-31"]

# Suppose you have daily returns in a column named 'DailyReturn':
in_ret = in_sample_df["DailyReturn"]
out_ret = out_sample_df["DailyReturn"]

in_sharpe = sharpe_ratio(in_ret, risk_free_annual=0.02)
out_sharpe = sharpe_ratio(out_ret, risk_free_annual=0.02)

results = {
    "InSampleSharpe": [in_sharpe],
    "OutSampleSharpe": [out_sharpe],
}
results_df = pd.DataFrame(results)
print("Results:\n", results_df)

# Cell 5: Export
to_excel(results_df, config["excel_path"])
to_csv(results_df, config["csv_path"])
to_json(results_df, config["json_path"])

print("Exports completed!")


</details>
Note that Cells 4 and onward in your real code will have more logic around volatility adjustments, manager selection, etc. The key is that you no longer need to define your metrics or reading logic inline: that code is in trendlib/.

7) Validation Steps
Create the trendlib/ folder in your repo alongside notebooks/.

Add each .py file with the code above.

Adjust config.py to match your actual file paths, column names, or date logic.

In Vol_Adj_Trend_Analysis1.2.TrEx.ipynb, replace old inline code with imports from trendlib/.

Run the notebook.

Confirm you get the same results as before or make small fixes if variable names differ.



Final Thoughts
This refactoring approach should keep all the same variable names, column references, and core logic you had in Vol_Adj_Trend_Analysis1.2.TrEx.ipynb, just distributed among modules for maintainability.

Where I wrote placeholders (like # Insert your original dropna or merges here), be sure to copy the matching lines from your notebook if you rely on them.

The end result is a clean, modular codebase that you can easily extend for Phase 2 (multi-period simulation, manager thresholds, etc.) and Phase 3 (Monte Carlo).

If you run into any issues—like references to variables that moved or missing columns—just let me know and I can suggest how to reconcile them. But this should be a solid template to get you started.